# Power System Load Flow Analysis
[Load flow analysis](https://nptel.ac.in/content/storage2/courses/108104051/ui/Course_home-4.htm) is a steady state analysis method which aims at identifying the flow of power from one node (or bus) to other in an interconnected network. A better (or more precise) term to use for this analysis method would be Power Flow Anlaysis. The following codes are written to implement these power flow equation solvers, given that some of the system parameters are known. The numerical methods used here to solve the Power Flow equations are `Gauss-Seidel Method` and `Newton-Raphson Method`. The results are further verified by modeling the system and solving its various equations with the help of [Pandapower](http://www.pandapower.org/).

### Contents: 

1.   Input Specification Files 
2.   Solver - Gauss Seidel Method
3.   Solver - Newton Raphson Method
4.   Example - [Three Bus System](http://www.fglongatt.org/Test_Systems/FGL3bus.html) (Synthetic Network)
5.   Example - [Four Bus System](https://matpower.org/docs/ref/matpower5.0/case4gs.html) (Synthetic Network)
6.   Example - [Five Bus System](https://nptel.ac.in/content/storage2/courses/108104051/ui/Course_home-4.htm) (Synthetic Network)
7.   Example - [Kundur's Two Area Network](https://link.springer.com/content/pdf/bbm%3A978-3-319-02393-9%2F1.pdf) (Crucial from Stability point of view)
8.   Example - [IEEE 14 Bus Network](https://icseg.iti.illinois.edu/ieee-14-bus-system/) (Non-synthetic Network) 



# Input Specification Files
### File Format 

*   linedata.csv
>From, To, R, X, Y_2
>1. From and To are bus numbers.
>2. Resistance and Impedance values are in per unit notation. 
*   busdata.csv
>Bus No, Bus Type, Pg, Qg, Pd, Qd, |V|, delta, Qmin, Qmax
>1. Bus Type is 1 for Slack bus, 2 for PV bus, 3 for PQ bus
>2. Pg, Qg, Pd and Qd are known (or assumed as 0) values of the power generated and power demand at each node
>3. The Power values are to be mentioned in per unit notation.
>4. |V| and delta are the initial values of the voltage magnitude and phase. 
>5. |V| is in per unit notation.
>6. Qmin and Qmax are bus ratings, and in case a PV bus violates these limits, it is further considered as a PQ bus by the algorithm to proceed. Also, the Q values are floored to either Qmin or Qmax depending on the nature of violation.


In [ ]:
import os

dir = os.getcwd()
files = os.listdir(dir)

linedata_files = []
busdata_files = []
for f in files:
    if ((f[:9] == "linedata_") and (f[-3:]=="csv")):
        linedata_files.append(f)
for f in files:
    if ((f[:8] == "busdata_") and (f[-3:]=="csv")):
        busdata_files.append(f)

#@title 
#@markdown Choose input specification files
linedata = "linedata_1.csv" #@param ["linedata_1.csv", "linedata_2.csv", "linedata_3.csv"] {allow-input: true}
busdata = "busdata_1.csv" #@param ["busdata_1.csv", "busdata_2.csv", "busdata_3.csv"] {allow-input: true}

linedata_file = linedata
busdata_file  = busdata

# Solver - Gauss Seidel Method

## Implementation:
1. The nodes of the power system are modeled as buses, and their connections as lines. 
2. For each of the nodes (except Slack node) if it PV type
>1. Q value is calculted first, with the P and V values from previous iterations
>2. If Q_calculated violates the limits, it is floored.
>3. Now, once Q is determined for the iteration, V value is computed with it.
>4. However, in case of PV bus, the magnitude of V is preserved, by accomodating a correction step.
>5. This process is iterated unless convergence (i.e. V values do not differ largely between two consecutive iterations) is acheieved. 

Execute the following section to get result with Gauss Seidel Method.

In [ ]:
#@title Solver - Gauss-Seidel Method 
import pandas as pd
import numpy as np

#import argparse
##Command
##python gauss_seidel.py -ld linedata.csv -bd busdata.csv
##Uncomment this section to use command-line options
#parser = argparse.ArgumentParser()
#parser.add_argument("-ld", "--linedata", help = "Csv file with Linedata")
#parser.add_argument("-bd", "--busdata", help = "Csv file with Busdata")
#args = parser.parse_args()
#linedata_file = args.linedata
#busdata_file  = args.busdata

#scaling factor for faster convergence of the algorithm 
alpha   = 1.6

#Reading line data from csv file 
linedata = pd.read_csv(linedata_file)

#Reading bus data from csv file 
busdata = pd.read_csv(busdata_file)

#Resistance and Impedance matrices
R       = np.array(linedata["R"], dtype=float)
X       = np.array(linedata["X"], dtype=float)
B       = np.array(1j * linedata["Y_2"], dtype=complex)
Z       = np.array(R + 1j * X, dtype=complex)
Y       = 1/Z
nline   = len(linedata)
nbus    = len(busdata)

#Creating the Ybus matrix 
From    = np.array(linedata['From'], dtype=int) - 1
To      = np.array(linedata['To'], dtype=int) - 1
Ybus    = np.zeros([nbus, nbus], dtype=complex)
for i in range(nline):
    Ybus[From[i],To[i]]     = Ybus[From[i],To[i]] - Y[i]
    Ybus[To[i],From[i]]     = Ybus[From[i],To[i]]
    Ybus[From[i],From[i]]   = Ybus[From[i],From[i]] + Y[i] + B[i]
    Ybus[To[i],To[i]]       = Ybus[To[i],To[i]] + Y[i] + B[i]

#Reading the power and votlage specifications of the buses 
bus_type = np.array(busdata['Bus Type'])
Pg      = np.array(busdata['Pg'])
Qg      = np.array(busdata['Qg'])
Pd      = np.array(busdata['Pd'])
Qd      = np.array(busdata['Qd'])
Vmag    = np.array(busdata['|V|'])
delta   = np.array(busdata['delta'])
Qmin    = np.array(busdata['Qmin'])
Qmax    = np.array(busdata['Qmax'])

#Initialising power and voltage values 
V   = Vmag * (np.cos(delta) + 1j * np.sin(delta))
P   = Pg - Pd
Q   = Qg - Qd
error = 1

i   = 1
max_iter    = 50
err_threshold   = 0.000001

while ((i < max_iter) and (error > err_threshold)):
    #Bus 1 is kept to be the slack bus thus no calculation required for it 
    
    dV  = np.zeros(nbus, dtype=complex)
    for n in range(1,nbus):
        
        if(bus_type[n] == 2): 
        #PV bus 
            I   = sum(np.multiply(Ybus[n,:],V))
            S   = np.conjugate(V[n]) * I
            Q[n] = -1*S.imag

            A   = I - Ybus[n,n]*V[n]

            if((Q[n] > Qmin[n]) and (Q[n] < Qmax[n])) : 
                Vnew    = (1/Ybus[n,n]) * ((P[n] - 1j*Q[n])/np.conjugate(V[n]) - A)
                Vnew    = Vmag[n] * (Vnew / abs(Vnew))
                #Magnitude correction
                dV[n]   = Vnew - V[n]
                V[n]    = Vnew
            
            else:
            #Treated as PQ bus
                if(Q[n] < Qmin[n]): 
                    Q[n] = Qmin[n]
                elif(Q[n] > Qmax[n]):
                    Q[n] = Qmax[n]
            
                Vnew    = (1/Ybus[n,n]) * ((P[n] - 1j*Q[n])/np.conjugate(V[n]) - A)
                Vnew    = V[n] + alpha * (Vnew - V[n])
                dV[n]   = Vnew - V[n]
                V[n]    = Vnew

        elif(bus_type[n] == 3):
        #PQ bus
            I = sum(np.multiply(Ybus[n,:],V))
            A = I - Ybus[n,n]*V[n]

            Vnew    = (1/Ybus[n,n]) * ((P[n] - 1j*Q[n])/np.conjugate(V[n]) - A)
            Vnew    = V[n] + alpha * (Vnew - V[n])
            dV[n]   = Vnew - V[n]
            V[n]    = Vnew

    error = np.linalg.norm(dV)
    i = i + 1

comp_data = np.zeros([nbus,10])

comp_data[:,0] = np.abs(V)
comp_data[:,1] = np.angle(V, deg=True)

#Given data
comp_data[:,2] = Pg
comp_data[:,3] = Qg
comp_data[:,4] = Pd
comp_data[:,5] = Qd

#Power calculations at each bus
S = np.multiply(np.conjugate(V), np.matmul(Ybus, V))
comp_data[:,6] = S.real
comp_data[:,7] = -1*S.imag

#Power loss calculations at each bus
L = (Pg - Pd) - 1j*(Qg - Qd) - S
comp_data[:,8] = L.real
comp_data[:,9] = L.imag

power_flow = pd.DataFrame(data=comp_data, index=np.arange(nbus), columns=['Vmag','Vang','Pg','Qg','Pd','Qd','Pcalc','Qcalc','Ploss','Qloss'])

print("Load flow analysis with Gauss Seidel Method")
print("# iter for convergence: " + str(i))
print("Error: " + str(error))
print(power_flow[['Vmag','Vang']])
print(power_flow[['Pg', 'Qg', 'Pd', 'Qd']])
print(power_flow[['Pcalc', 'Qcalc', 'Ploss', 'Qloss']])


# Solver - Newton Raphson Method

## Implementation:
1. The bus and line data are given to the algorithm.
2. At every iteration the Jacobian matrix is created with the current values of P, Q and V.
3. This is an indirect way of calculating the derivative, which is further multiplied with a short step size and added to the current values. 
4. The algorithm iterates till convergence is achieved. 
5. The jacobian matrix is formed as per [this](https://nptel.ac.in/content/storage2/courses/108104051/ui/Course_home-4.htm).

Execute the following section to get result with Newton Raphson Method.

In [ ]:
#@title Solver - Newton-Raphson Method
#@markdown Scaling factor for correction (Faster Convergence)
a = 1.6 #@param {type:"slider", min:1, max:2, step:0.1}
import pandas as pd
import numpy as np

#import argparse
##Command
##python gauss_seidel.py -ld linedata.csv -bd busdata.csv
##Uncomment this section to use command-line options
#parser = argparse.ArgumentParser()
#parser.add_argument("-ld", "--linedata", help = "Csv file with Linedata")
#parser.add_argument("-bd", "--busdata", help = "Csv file with Busdata")
#args = parser.parse_args()
#linedata_file = args.linedata
#busdata_file  = args.busdata

#Reading line data from csv file 
linedata = pd.read_csv(linedata_file)

#Reading bus data from csv file 
busdata = pd.read_csv(busdata_file)

#Resistance and Impedance matrices
R       = np.array(linedata["R"],           dtype=float)
X       = np.array(linedata["X"],           dtype=float)
B       = np.array(1j * linedata["Y_2"],    dtype=complex)
Z       = np.array(R + 1j * X,              dtype=complex)
Y       = 1/Z
nline   = len(linedata)
nbus    = len(busdata)

#Creating the Ybus matrix 
From    = np.array(linedata['From'],    dtype=int) - 1
To      = np.array(linedata['To'],      dtype=int) - 1
Ybus    = np.zeros([nbus, nbus],        dtype=complex)
for i in range(nline):
    Ybus[From[i],To[i]]     = Ybus[From[i],To[i]]   - Y[i]
    Ybus[To[i],From[i]]     = Ybus[From[i],To[i]]
    Ybus[From[i],From[i]]   = Ybus[From[i],From[i]] + Y[i] + B[i]
    Ybus[To[i],To[i]]       = Ybus[To[i],To[i]]     + Y[i] + B[i]

G       = Ybus.real
B       = Ybus.imag

#Reading the power and votlage specifications of the buses 
bus_type = np.array(busdata['Bus Type'])
Pg      = np.array(busdata['Pg'],       dtype=float)
Qg      = np.array(busdata['Qg'],       dtype=float)
Pd      = np.array(busdata['Pd'],       dtype=float)
Qd      = np.array(busdata['Qd'],       dtype=float)
Vmag    = np.array(busdata['|V|'],      dtype=float)
delta   = np.array(busdata['delta'],    dtype=float)
Qmin    = np.array(busdata['Qmin'],     dtype=float)
Qmax    = np.array(busdata['Qmax'],     dtype=float)
pq_bus  = []
i       = 0
for t in bus_type:
    if(t == 3):
        pq_bus.append(i)
    i = i + 1
n0      = len(pq_bus)

#Initialising power and voltage values 
V   = Vmag * (np.cos(delta) + 1j * np.sin(delta))
P   = Pg - Pd
Q   = Qg - Qd
error = 1

i   = 1
max_iter    = 50
err_threshold   = 0.000001

while ((i < max_iter) and (error > err_threshold)):
    
    #P and Q calculation
    Scalc = np.multiply(np.conjugate(V), np.matmul(Ybus,V))
    Pcalc = Scalc.real
    Qcalc = -1*Scalc.imag

    #Jacobian matrix formation
    J11 = np.zeros([nbus-1,nbus-1])
    for n in range(0,nbus-1):
        for m in range(0,nbus-1):
            if(n != m):
                J11[n,m] = -1 * abs(Ybus[n+1,m+1]) * Vmag[n+1] * Vmag[m+1] * np.sin(np.angle(Ybus[n+1,m+1]) + delta[m+1] - delta[n+1])
            else:
                J11[n,m] = -1*Qcalc[n+1] - (Vmag[n+1] * Vmag[n+1] * B[n+1,n+1]) 
    
    J21 = np.zeros([n0,nbus-1])
    for n in range(0,n0):
        for m in range(0,nbus-1):
            bn = pq_bus[n]
            if(bn != m+1):
                J21[n,m] = -1 * abs(Ybus[bn,m+1]) * Vmag[bn] * Vmag[m+1] * np.cos(np.angle(Ybus[bn,m+1]) + delta[m+1] - delta[bn])
            else:
                J21[n,m] = Pcalc[bn] - (Vmag[bn] * Vmag[bn] * G[bn,bn])

    J12 = np.zeros([nbus-1,n0])
    for n in range(0,nbus-1):
        for m in range(0,n0):
            bm = pq_bus[m]
            if(bm != n+1):            
                J12[n,m] = abs(Ybus[n+1,bm]) * Vmag[n+1] * Vmag[bm] * np.cos(np.angle(Ybus[n+1,bm]) + delta[bm] - delta[n+1])
            else:
                J12[n,m] = Pcalc[bm] + (Vmag[bm] * Vmag[bm] * G[bm,bm])

    J22 = np.zeros([n0,n0])
    for n in range(0,n0):
        for m in range(0,n0):
            bn = pq_bus[n]
            bm = pq_bus[m]
            if(bn != bm):
                J22[n,m] = -1 * abs(Ybus[bn,bm]) * Vmag[bn] * Vmag[bm] * np.sin(np.angle(Ybus[bn,bm]) + delta[bm] - delta[bn])
            else:
                J22[n,m] = Qcalc[bn] - (Vmag[bn] * Vmag[bn] * B[bn,bn]) 

    J = np.zeros([nbus+n0-1,nbus+n0-1], dtype=complex)
    J[:nbus-1,:nbus-1]  = J11
    J[:nbus-1,nbus-1:]  = J12
    J[nbus-1:,:nbus-1]  = J21
    J[nbus-1:,nbus-1:]  = J22

    #Calculating difference with updated value
    dP              = P - Pcalc
    dQ              = Q - Qcalc
    dPQ             = np.zeros(nbus+n0-1)
    dPQ[:nbus-1]    = dP[1:]
    n = 0
    m = 0
    for n in range(nbus):
        if(bus_type[n] == 3):
            dPQ[nbus-1+m] = dQ[n]
            m = m + 1

    #Correction in the Voltage magnitudes and angles 
    corr = np.matmul( np.linalg.inv(J), dPQ)
    corr = corr.real
    for n in range(1,nbus):
        delta[n]        = delta[n] + a * corr[n-1]
    for n in range(n0):
        Vmag[pq_bus[n]] = Vmag[pq_bus[n]] *(1 + a * corr[nbus-1+n])
    
    dV      = Vmag * (np.cos(delta) + 1j*np.sin(delta)) - V
    V       = Vmag * (np.cos(delta) + 1j * np.sin(delta))
    
    error   = np.linalg.norm(dV)
    i       = i + 1

comp_data = np.zeros([nbus,10])

comp_data[:,0] = Vmag
comp_data[:,1] = delta*(180/(22/7))

#Given data
comp_data[:,2] = Pg
comp_data[:,3] = Qg
comp_data[:,4] = Pd
comp_data[:,5] = Qd

#Power calculations at each bus
S = np.multiply(np.conjugate(V), np.matmul(Ybus, V))
comp_data[:,6] = S.real
comp_data[:,7] = -1*S.imag

#Power loss calculations at each bus
L = (Pg - Pd) - 1j*(Qg - Qd) - S
comp_data[:,8] = L.real
comp_data[:,9] = L.imag

power_flow = pd.DataFrame(data=comp_data, index=np.arange(nbus), columns=['Vmag','Vang','Pg','Qg','Pd','Qd','Pcalc','Qcalc','Ploss','Qloss'])

print("Load flow analysis with Newton Raphson Method")
print("# iter for convergence: " + str(i))
print("Error: " + str(error))
print(power_flow[['Vmag','Vang']])
print(power_flow[['Pg', 'Qg', 'Pd', 'Qd']])
print(power_flow[['Pcalc', 'Qcalc', 'Ploss', 'Qloss']])


In [ ]:
#@title Run to install Pandapower
#@markdown Needs to be run only once
!pip install pandapower

# Example - Three Bus System 

[Source](http://www.fglongatt.org/Test_Systems/FGL3bus.html)

<img src="http://www.fglongatt.org/Test_Systems/images/FGL_3bus_Test_System.png" width="350" height="200">


In [ ]:
#@title Three Bus System Modeled in Pandapower
import pandapower as pp
net=pp.create_empty_network(f_hz=60,sn_mva=100)

# create bus
bus1=pp.create_bus(net,vn_kv=115)
bus2=pp.create_bus(net,vn_kv=115)
bus3=pp.create_bus(net,vn_kv=115)

#create line
pp.create_impedance(net,from_bus=bus1,to_bus=bus2,rft_pu=0.04,xft_pu=0.02,sn_mva=100)
pp.create_impedance(net,from_bus=bus1,to_bus=bus3,rft_pu=0.03,xft_pu=0.01,sn_mva=100)
pp.create_impedance(net,from_bus=bus2,to_bus=bus3,rft_pu=0.025,xft_pu=0.0125,sn_mva=100)

# add the generator
pp.create_ext_grid(net,bus=bus1,vm_pu=1.05,va_degree=0,max_q_mvar=250)
pp.create_gen(net,bus=bus3,sn_mva=100,p_mw=200,vm_pu=1.04,max_q_mvar=40,min_q_mvar=-20)

# add the load
pp.create_load(net,bus=bus2,p_mw=400,q_mvar=200)

## run the power flow
#pp.runpp(net,algorithm='gs',enforce_q_lims=True)
## Results for the bus
#print(net.res_bus)

# run the power flow
pp.runpp(net,algorithm='nr',enforce_q_lims=True)

# Results for the bus
print(net.res_bus)

# Example - Four Bus System

[Source](https://matpower.org/docs/ref/matpower5.0/case4gs.html)

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRRTiDaUaEOeHXjwMHkBpS6_Woor1IgbER77A&usqp=CAU" width="350" height="200">

In [ ]:
#@title Four Bus System Example
import pandapower as pp
import pandas as pd
# Create the network/initialize the network frquency/base
net=pp.create_empty_network(f_hz=50,sn_mva=100)

# create the bus
bus1=pp.create_bus(net,vn_kv=230)
bus2=pp.create_bus(net,vn_kv=230)
bus3=pp.create_bus(net,vn_kv=230)
bus4=pp.create_bus(net,vn_kv=230)

# create the line
line1=pp.create_line_from_parameters(net,from_bus=bus1,to_bus=bus2,length_km=1,r_ohm_per_km=5.33232,x_ohm_per_km=26.6616,c_nf_per_km=514.23,max_i_ka=10)
line2=pp.create_line_from_parameters(net,from_bus=bus1,to_bus=bus3,length_km=1,r_ohm_per_km=3.93576,x_ohm_per_km=19.6788,c_nf_per_km=388.808,max_i_ka=10)
line3=pp.create_line_from_parameters(net,from_bus=bus2,to_bus=bus4,length_km=1,r_ohm_per_km=3.93576,x_ohm_per_km=19.6788,c_nf_per_km=388.808,max_i_ka=10)
line4=pp.create_line_from_parameters(net,from_bus=bus3,to_bus=bus4,length_km=1,r_ohm_per_km=6.72888,x_ohm_per_km=33.6444,c_nf_per_km=639.652,max_i_ka=10)

# add the generator
pp.create_ext_grid(net,bus=bus1,vm_pu=1,va_degree=0)
pp.create_gen(net,bus=bus4,p_mw=318,vm_pu=1.02)

# add the load
pp.create_load(net,bus=bus1,p_mw=50,q_mvar=30)
pp.create_load(net,bus=bus2,p_mw=170,q_mvar=105.35)
pp.create_load(net,bus=bus3,p_mw=200,q_mvar=123.94)
pp.create_load(net,bus=bus4,p_mw=80,q_mvar=49.58)

##Run Load Flow Analysis
#pp.runpp(net,algorithm='gs',enforce_q_lims=True)
##Report 
#print(net.res_bus)
#print(net.res_line)

#Run Load Flow Analysis
pp.runpp(net,algorithm='nr',enforce_q_lims=True)
#Report 
print(net.res_bus)
print(net.res_line)

# Example - Five Bus System

[Source](https://nptel.ac.in/content/storage2/courses/108104051/ui/Course_home-4.htm)

<img src="https://nptel.ac.in/content/storage2/courses/108104051/chapter_4/images/image024.jpg" width="350" height="200">

In [ ]:
#@title Five Bus System Example
import pandapower as pp
import pandas as pd
# Create the network/initialize the network frquency/base
net=pp.create_empty_network(f_hz=50,sn_mva=100)

# create the bus
bus1=pp.create_bus(net,vn_kv=230)
bus2=pp.create_bus(net,vn_kv=230)
bus3=pp.create_bus(net,vn_kv=230)
bus4=pp.create_bus(net,vn_kv=230)
bus5=pp.create_bus(net,vn_kv=230)

# create the line
line1=pp.create_line_from_parameters(net,from_bus=bus1,to_bus=bus2,length_km=1,r_ohm_per_km=10.58,x_ohm_per_km=52.9,c_nf_per_km=15.87,max_i_ka=10)
line2=pp.create_line_from_parameters(net,from_bus=bus1,to_bus=bus5,length_km=1,r_ohm_per_km=26.45,x_ohm_per_km=132.25,c_nf_per_km=10.58,max_i_ka=10)
line3=pp.create_line_from_parameters(net,from_bus=bus2,to_bus=bus3,length_km=1,r_ohm_per_km=21.16,x_ohm_per_km=105.8,c_nf_per_km=13.22,max_i_ka=10)
line4=pp.create_line_from_parameters(net,from_bus=bus2,to_bus=bus5,length_km=1,r_ohm_per_km=26.45,x_ohm_per_km=132.25,c_nf_per_km=10.58,max_i_ka=10)
line5=pp.create_line_from_parameters(net,from_bus=bus3,to_bus=bus4,length_km=1,r_ohm_per_km=26.45,x_ohm_per_km=132.25,c_nf_per_km=10.58,max_i_ka=10)
line6=pp.create_line_from_parameters(net,from_bus=bus3,to_bus=bus5,length_km=1,r_ohm_per_km=42.32,x_ohm_per_km=211.6,c_nf_per_km=5.29,max_i_ka=10)
line7=pp.create_line_from_parameters(net,from_bus=bus4,to_bus=bus5,length_km=1,r_ohm_per_km=52.9,x_ohm_per_km=264.5,c_nf_per_km=39.675,max_i_ka=10)

# add the generator
pp.create_ext_grid(net,bus=bus1,vm_pu=1.05,va_degree=0)
pp.create_gen(net,bus=bus5,p_mw=48,vm_pu=1.02)

# add the load
pp.create_load(net,bus=bus2,p_mw=96,q_mvar=62)
pp.create_load(net,bus=bus3,p_mw=35,q_mvar=14)
pp.create_load(net,bus=bus4,p_mw=16,q_mvar=8)
pp.create_load(net,bus=bus5,p_mw=24,q_mvar=11)

#Run Load Flow Analysis
pp.runpp(net,algorithm='gs',enforce_q_lims=True)
#Report
print(net.res_bus)
print(net.res_line)

#Run Load Flow Analysis
pp.runpp(net,algorithm='nr',enforce_q_lims=True)
#Report
print(net.res_bus)
print(net.res_line)

# Kundur's Two Area Network

Note: The Generator ratings are 900MVA and 20kV, while transformer rating is 900MVA and 230kV/20kV. The high voltage side is taken on the bus side, as the step up transformer is used before transmitting power, to save on the Isq R losses in the line. 

Further, the impedances are converted into their per km values, from per unit values given, by taking the base values as 900MVA and 230kV (as the lines are connected to the HV side of transformer which is at 230kV). 

<img src="https://www.fglongatt.org/Test_Systems/images/2-Areas_Kundur.png" width="500" height="300">

1. The compensators present at the bus 8 and 9 are modeled as shunt_with_capacitor in the Pandapower model, and the given imaginary power values are used to characterize it. 

```
# pp.create_shunt_as_capacitor(net,   bus=bus7,   q_mvar=200, loss_factor=0)
```
2. The buses at different voltage levels are connected with transformer. 

```
# t1 = pp.create_transformer_from_parameters(net, hv_bus=bus5,  lv_bus=bus1, vn_hv_kv=230, vn_lv_kv=20, sn_mva=900, vkr_percent=10, vk_percent=10, pfe_kw=0.01, i0_percent=0.01)
```

3. Run the following section to see the Load flow analysis results.


In [ ]:
#@title Kundur's Two Area Network
import pandapower as pp
# Create the network/initialize the network frquency/base
net=pp.create_empty_network(f_hz=50, sn_mva=900)

# create the bus
bus1	=	pp.create_bus(net,vn_kv=20)
bus2	=	pp.create_bus(net,vn_kv=20)
bus3	=	pp.create_bus(net,vn_kv=20)
bus4	=	pp.create_bus(net,vn_kv=20)
bus5	=	pp.create_bus(net,vn_kv=230)
bus6	=	pp.create_bus(net,vn_kv=230)
bus7	=	pp.create_bus(net,vn_kv=230)
bus8	=	pp.create_bus(net,vn_kv=230)
bus9	=	pp.create_bus(net,vn_kv=230)
bus10	=	pp.create_bus(net,vn_kv=230)
bus11	=	pp.create_bus(net,vn_kv=230)

# create the line
line1  =   pp.create_line_from_parameters(net, from_bus=bus5,   to_bus=bus6,   length_km=25,   r_ohm_per_km=0.00587,   x_ohm_per_km=0.0587,   c_nf_per_km=94.77,    max_i_ka=1)
line2  =   pp.create_line_from_parameters(net, from_bus=bus10,  to_bus=bus11,  length_km=25,   r_ohm_per_km=0.00587,   x_ohm_per_km=0.0587,   c_nf_per_km=94.77,    max_i_ka=1)
line3  =   pp.create_line_from_parameters(net, from_bus=bus6,   to_bus=bus7,   length_km=10,   r_ohm_per_km=0.00587,   x_ohm_per_km=0.0587,   c_nf_per_km=94.77,    max_i_ka=1)
line4  =   pp.create_line_from_parameters(net, from_bus=bus9,   to_bus=bus10,  length_km=10,   r_ohm_per_km=0.00587,   x_ohm_per_km=0.0587,   c_nf_per_km=94.77,    max_i_ka=1)
line5  =   pp.create_line_from_parameters(net, from_bus=bus7,   to_bus=bus8,   length_km=110,  r_ohm_per_km=0.00587,   x_ohm_per_km=0.0587,   c_nf_per_km=94.77,    max_i_ka=1)
line6  =   pp.create_line_from_parameters(net, from_bus=bus7,   to_bus=bus8,   length_km=110,  r_ohm_per_km=0.00587,   x_ohm_per_km=0.0587,   c_nf_per_km=94.77,    max_i_ka=1)
line7  =   pp.create_line_from_parameters(net, from_bus=bus8,   to_bus=bus9,   length_km=110,  r_ohm_per_km=0.00587,   x_ohm_per_km=0.0587,   c_nf_per_km=94.77,    max_i_ka=1)
line8  =   pp.create_line_from_parameters(net, from_bus=bus8,   to_bus=bus9,   length_km=110,  r_ohm_per_km=0.00587,   x_ohm_per_km=0.0587,   c_nf_per_km=94.77,    max_i_ka=1)

# add the generator
pp.create_ext_grid(net, bus=bus1,   vm_pu=1.03,	va_degree=0)
pp.create_gen(net,      bus=bus2,   p_mw=700,   vm_pu=1.01)
pp.create_gen(net,      bus=bus3,   p_mw=719,   vm_pu=1.03)
pp.create_gen(net,      bus=bus4,   p_mw=700,   vm_pu=1.01)

# add the load
pp.create_load(net,     bus=bus7,   p_mw=967,  q_mvar=100)
pp.create_load(net,     bus=bus9,   p_mw=1767, q_mvar=100)
pp.create_shunt_as_capacitor(net,   bus=bus7,   q_mvar=200, loss_factor=0)
pp.create_shunt_as_capacitor(net,   bus=bus9,   q_mvar=350, loss_factor=0)

#adding transformers and compensators
t1 = pp.create_transformer_from_parameters(net, hv_bus=bus5,  lv_bus=bus1, vn_hv_kv=230, vn_lv_kv=20, sn_mva=900, vkr_percent=10, vk_percent=10, pfe_kw=0.01, i0_percent=0.01)
t2 = pp.create_transformer_from_parameters(net, hv_bus=bus6,  lv_bus=bus2, vn_hv_kv=230, vn_lv_kv=20, sn_mva=900, vkr_percent=10, vk_percent=10, pfe_kw=0.01, i0_percent=0.01)
t4 = pp.create_transformer_from_parameters(net, hv_bus=bus10, lv_bus=bus4, vn_hv_kv=230, vn_lv_kv=20, sn_mva=900, vkr_percent=10, vk_percent=10, pfe_kw=0.01, i0_percent=0.01)
t3 = pp.create_transformer_from_parameters(net, hv_bus=bus11, lv_bus=bus3, vn_hv_kv=230, vn_lv_kv=20, sn_mva=900, vkr_percent=10, vk_percent=10, pfe_kw=0.01, i0_percent=0.01)
pp.create_switch(net, bus=bus5, element=t1, et="t", type="DS")
pp.create_switch(net, bus=bus6, element=t2, et="t", type="DS")
pp.create_switch(net, bus=bus10, element=t4, et="t", type="DS")
pp.create_switch(net, bus=bus11, element=t3, et="t", type="DS")

##Run the following section to diagnose errors in the modeled system
#errors = pp.diagnostic(net, report_style='detailed')
#print(errors)

#pp.runpp(net,algorithm='gs', max_iteration=5000)
#print(net.res_bus)
#print(net.res_line)

pp.runpp(net,algorithm='nr',init='flat')
print(net.res_bus)
print(net.res_line)

# Example - IEEE 14 Bus System

<img src="https://www.researchgate.net/profile/Prasenjit-Dey/publication/322140152/figure/fig2/AS:583057972436992@1516023261578/Single-line-diagram-for-IEEE-14-bus-system.png" width="500" height="300">

The data for the system is taken from [here](https://www.researchgate.net/profile/Mohamed_Mourad_Lafifi/post/Datasheet_for_5_machine_14_bus_ieee_system2/attachment/59d637fe79197b8077995409/AS%3A395594356019200%401471328452063/download/DATA+SHEETS+FOR+IEEE+14+BUS+SYSTEM+19_appendix.pdf).
It is verified that the values are taken with a consistent base values for Voltage and Power. 

Note: The components added are similar to that in the Two area network - i.e. generators connected at some of the buses, transformers used for connecting different voltage levels and shunt used for modelling the compensator at Bus9 (L9).

Run the following section to see the Load Flow Analysis Report.

In [ ]:
#@title IEEE 14 Bus System
import pandapower as pp
# Create the network/initialize the network frquency/base
net=pp.create_empty_network(f_hz=50)

# create the bus
bus1	=	pp.create_bus(net,  vn_kv=135,     max_vm_pu=1.04,      min_vm_pu=0.96)
bus2	=	pp.create_bus(net,  vn_kv=135,     max_vm_pu=1.04,      min_vm_pu=0.96)
bus3	=	pp.create_bus(net,  vn_kv=135,     max_vm_pu=1.04,      min_vm_pu=0.96)
bus4	=	pp.create_bus(net,  vn_kv=135,     max_vm_pu=1.04,      min_vm_pu=0.96)
bus5	=	pp.create_bus(net,  vn_kv=135,     max_vm_pu=1.04,      min_vm_pu=0.96)
bus6	=	pp.create_bus(net,  vn_kv=0.208,   max_vm_pu=1.04,      min_vm_pu=0.96)
bus7	=	pp.create_bus(net,  vn_kv=14,      max_vm_pu=1.04,      min_vm_pu=0.96)
bus8	=	pp.create_bus(net,  vn_kv=12,      max_vm_pu=1.04,      min_vm_pu=0.96)
bus9	=	pp.create_bus(net,  vn_kv=0.208,   max_vm_pu=1.04,      min_vm_pu=0.96)
bus10	=	pp.create_bus(net,  vn_kv=0.208,   max_vm_pu=1.04,      min_vm_pu=0.96)
bus11	=	pp.create_bus(net,  vn_kv=0.208,   max_vm_pu=1.04,      min_vm_pu=0.96)
bus12	=	pp.create_bus(net,  vn_kv=0.208,   max_vm_pu=1.04,      min_vm_pu=0.96)
bus13	=	pp.create_bus(net,  vn_kv=0.208,   max_vm_pu=1.04,      min_vm_pu=0.96)
bus14	=	pp.create_bus(net,  vn_kv=0.208,   max_vm_pu=1.04,      min_vm_pu=0.96)

# create the line
line1  =   pp.create_line_from_parameters(net, from_bus=bus1,   to_bus=bus2,   length_km=1,  r_ohm_per_km=3.5320,    x_ohm_per_km=10.78373,   c_nf_per_km=768.48,    max_i_ka=42.33)
line2  =   pp.create_line_from_parameters(net, from_bus=bus1,   to_bus=bus5,   length_km=1,  r_ohm_per_km=9.8469,    x_ohm_per_km=40.64904,   c_nf_per_km=716.08,    max_i_ka=42.33)
line3  =   pp.create_line_from_parameters(net, from_bus=bus2,   to_bus=bus3,   length_km=1,  r_ohm_per_km=8.5639,    x_ohm_per_km=36.08000,   c_nf_per_km=637.49,    max_i_ka=42.33)
line4  =   pp.create_line_from_parameters(net, from_bus=bus2,   to_bus=bus4,   length_km=1,  r_ohm_per_km=10.590,    x_ohm_per_km=32.13432,   c_nf_per_km=494.86,    max_i_ka=42.33)
line5  =   pp.create_line_from_parameters(net, from_bus=bus2,   to_bus=bus5,   length_km=1,  r_ohm_per_km=10.379,    x_ohm_per_km=31.68963,   c_nf_per_km=503.59,    max_i_ka=42.33)
line6  =   pp.create_line_from_parameters(net, from_bus=bus3,   to_bus=bus4,   length_km=1,  r_ohm_per_km=12.212,    x_ohm_per_km=31.17021,   c_nf_per_km=186.29,    max_i_ka=42.33)
line7  =   pp.create_line_from_parameters(net, from_bus=bus4,   to_bus=bus5,   length_km=1,  r_ohm_per_km=2.4330,    x_ohm_per_km=7.674547,   c_nf_per_km=0,         max_i_ka=42.33)
line8  =   pp.create_line_from_parameters(net, from_bus=bus6,   to_bus=bus11,  length_km=1,  r_ohm_per_km=0.00004,   x_ohm_per_km=0.000086,   c_nf_per_km=0,         max_i_ka=27500)
line9  =   pp.create_line_from_parameters(net, from_bus=bus6,   to_bus=bus12,  length_km=1,  r_ohm_per_km=0.00005,   x_ohm_per_km=0.000110,   c_nf_per_km=0,         max_i_ka=27500)
line10 =   pp.create_line_from_parameters(net, from_bus=bus6,   to_bus=bus13,  length_km=1,  r_ohm_per_km=0.00003,   x_ohm_per_km=0.000056,   c_nf_per_km=0,         max_i_ka=27500)
line11 =   pp.create_line_from_parameters(net, from_bus=bus9,   to_bus=bus10,   length_km=1,  r_ohm_per_km=0.00001,   x_ohm_per_km=0.000036,   c_nf_per_km=0,         max_i_ka=27500)
line12 =   pp.create_line_from_parameters(net, from_bus=bus9,   to_bus=bus14,  length_km=1,  r_ohm_per_km=0.00006,   x_ohm_per_km=0.000117,   c_nf_per_km=0,         max_i_ka=27500)
line13 =   pp.create_line_from_parameters(net, from_bus=bus10,  to_bus=bus11,  length_km=1,  r_ohm_per_km=0.00004,   x_ohm_per_km=0.000083,   c_nf_per_km=0,         max_i_ka=27500)
line14 =   pp.create_line_from_parameters(net, from_bus=bus12,  to_bus=bus13,  length_km=1,  r_ohm_per_km=0.00010,   x_ohm_per_km=0.000087,   c_nf_per_km=0,         max_i_ka=27500)
line15 =   pp.create_line_from_parameters(net, from_bus=bus13,  to_bus=bus14,  length_km=1,  r_ohm_per_km=0.00007,   x_ohm_per_km=0.000151,   c_nf_per_km=0,         max_i_ka=27500)

# add the generator
pp.create_ext_grid(net, bus=bus1,   vm_pu=1.06,	va_degree=0)
pp.create_gen(net,      bus=bus2,   p_mw=40,   vm_pu=1.045,     max_q_mvar=50,    min_q_mvar=-40)
pp.create_gen(net,      bus=bus3,   p_mw=0,    vm_pu=1.01,      max_q_mvar=40,    min_q_mvar=0)
pp.create_gen(net,      bus=bus6,   p_mw=0,    vm_pu=1.07,      max_q_mvar=24,    min_q_mvar=-6)
pp.create_gen(net,      bus=bus8,   p_mw=0,    vm_pu=1.09,      max_q_mvar=24,    min_q_mvar=-6)

# add the load
pp.create_load(net,     bus=bus2,    p_mw=21.7,   q_mvar=12.7)
pp.create_load(net,     bus=bus3,    p_mw=94.2,   q_mvar=19)
pp.create_load(net,     bus=bus4,    p_mw=47.8,   q_mvar=-3.9)
pp.create_load(net,     bus=bus5,    p_mw=7.6,    q_mvar=1.6)
pp.create_load(net,     bus=bus6,    p_mw=11.2,   q_mvar=7.5)
pp.create_load(net,     bus=bus9,    p_mw=29.5,   q_mvar=16.6)
pp.create_load(net,     bus=bus10,   p_mw=9,      q_mvar=5.8)
pp.create_load(net,     bus=bus11,   p_mw=3.5,    q_mvar=1.8)
pp.create_load(net,     bus=bus12,   p_mw=6.1,    q_mvar=1.6)
pp.create_load(net,     bus=bus13,   p_mw=13.5,   q_mvar=5.8)
pp.create_load(net,     bus=bus14,   p_mw=14.9,   q_mvar=5)

# add the shunt
pp.create_shunt_as_capacitor(net,   bus=bus9,   q_mvar=-19.8, loss_factor=0)

#adding transformers and compensators
t1 = pp.create_transformer_from_parameters(net, hv_bus=bus4,  lv_bus=bus7, vn_hv_kv=135, vn_lv_kv=14,    sn_mva=9900, vkr_percent=0, vk_percent=2070, pfe_kw=0, i0_percent=0)
t2 = pp.create_transformer_from_parameters(net, hv_bus=bus4,  lv_bus=bus9, vn_hv_kv=135, vn_lv_kv=0.208, sn_mva=9900, vkr_percent=0, vk_percent=5560, pfe_kw=0, i0_percent=0)
t3 = pp.create_transformer_from_parameters(net, hv_bus=bus5,  lv_bus=bus6, vn_hv_kv=135, vn_lv_kv=0.208, sn_mva=9900, vkr_percent=0, vk_percent=2494, pfe_kw=0, i0_percent=0)
t4 = pp.create_transformer_from_parameters(net, hv_bus=bus7,  lv_bus=bus8, vn_hv_kv=14,  vn_lv_kv=12,    sn_mva=9900, vkr_percent=0, vk_percent=1744, pfe_kw=0, i0_percent=0)
t5 = pp.create_transformer_from_parameters(net, hv_bus=bus7,  lv_bus=bus9, vn_hv_kv=14,  vn_lv_kv=0.208, sn_mva=9900, vkr_percent=0, vk_percent=1089, pfe_kw=0, i0_percent=0)

##Run the following section to diagnose errors in the modeled system
#errors = pp.diagnostic(net, report_style='detailed')
#print(errors)

#pp.runpp(net,algorithm='gs', max_iteration=5000)
#print(net.res_bus)
#print(net.res_line)

pp.runpp(net,algorithm='nr',init='flat')
print(net.res_bus)
print(net.res_line)